# Chapter 7: Ensemble Learning

## <font color=blue>Voting Classifiers </font>

Suppose we have a few classifiers like a Logistic Regression, an SVM classifier, a Random Forest classifier, a K-Nearest Neighbors classifier and etc, each achieving an acceptable accuracy, say 80%. A very simple way to create an even better classifier is to aggregate the predictions of each classifier and for each input, predict the class that gets the most votes. This majority-vote classifier is called a <i><b>hard voting classifier</b></i>. This new classifier often achieves a higher accuracy than those in the ensemle.

Ensemble methods like this one work best when the predictors in the ensemble are as independent from one another as possible. When all the predictors are trained on the same training set, they are not purely independent. One way to achieve rather diverse classifiers to use in ensemble is to train them using very different algorithms. This way, the chance that they will make different types of errors increases, improving the ensemble's accuracy.

A voting classfier can be imlpemented using scikit learn <code>VotingClassifier</code> class from <code>ensemble</code> package. To use hard voting method, <code>'hard'</code> argument must be used for <code>voting</code> parameter.
<br>
<br>
<code>log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()
voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')
voting_clf.fit(X_train, y_train)
</code>

If all the classifiers in ensemble are capable of computing a probability for each class (i.e. they have <code>predict_proba()</code> method), then we can tell scikit-learn to predict the class with the highest class probability, averaged over all the individual classifiers. This is called <i><b> soft voting</b></i>. It can achieve higher scores than hard voting since it gives more weight to highly confident votes. <code>voting="hard"</code> must be replaced with <code>voting="soft"</code> to use this method. Keep in mind that SVC doesn't compute probabalities by default, so we need to set its <code>probability</code> hyperparameter to <code>True</code>.

## <font color=blue>Bagging and Pasting</font>

Another way to get a diverse set of classifiers is to use the same training algorithm for all predictors and train them on defferent random subsets of the training set. When sampling is performed with replacement, it's called <i><b>bagging</b></i> (short for bootstrap aggregating). When sampling is done without replacement, it's called <i><b>pasting</b></i>. <br>
Both Bagging and Pasting allow taining instances to be sampled several time across multiple predictors, but only bagging allows training instances to be sampled several times for the same predictor.

Once all predictors are trained, the ensemble can make a prediction for a new instance by aggregating the predictions of all predictors. For classification, the aggreagation function is typically <i>statistical mode</i>. For regression, ensemble computes average of all predictions. Each predictor has more bias than if it was trained on the whole training set, but aggreagtion reduces both bias and variance.

Bagging and Pasting can be performed with scikit-learn API through <code>BaggingClassifier</code> class from ensemble package. This will perform Bagging bu default, but if you want to use pasting, set <code>bootsrtap</code> parameter to <code>false</code>.<br><br>
<code>bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)</code><br>
- Note that BaggingClassifier performs soft voting by default if the base predictor have <code>predict_proba()</code> method.

Bootstrapping introduces a bit more diversity in the subsets that each predictor is
trained on, so bagging ends up with a slightly higher bias than pasting; but the extra
diversity also means that the predictors end up being less correlated, so the ensemble’s
variance is reduced. Overall, bagging often results in better models, which explains why
it is generally preferred.

### Out-of-Bag Evaluation

Out of Bag evaluation is a way to validate a bagging ensemble model. In a bagging model, each base predictor is trained on approximately 67% of the training instances (in limit of large training set size). The remaining 37% of training instances that are not sampled are called <i>out-of-bag</i> instances. Note that they are not the same 37% for all predictors. Since a predictor never sees the oob instances during training, it can be evaluated on these instances, without the need for a seperate validation set.The ensemble itself can be evaluated by averaging all the oob scores. Also note that oob evaluation is different from cross-validation and it can be useful when the training set is not big enough to put aside some instances for CV. For more info head to: https://towardsdatascience.com/what-is-out-of-bag-oob-score-in-random-forest-a7fa23d710 <br>
In scikit learn, oob score is computed if you set the <code>oob_score</code> parameter to <code>True</code> when creating the BaggingClassifier object. The resulting evaluation can be accsessed through the <code>oob_score_</code> variable in bagging model.

## <font color=blue>Random Patches and Random Subspaces</font>

We can sample features too, just like what we did for training instances. Doing so can be useful when we are dealing with high-dimensional inputs like images. If we sample both instances and features, the moethod is called <i>Random Patches</i>. If we keep all the training instances but sample features, it's called <i>Random Subspaces</i> method. These two methods can be implemented with the same class from scikit-learn by setting two <code>max_features</code> and <code>bootstrap_features</code> parameters.<br>
Sampling features results in even more predictor diversity, trading a bit more bias for a
lower variance.

## <font color=blue>Random Forests</font>

Random Forest is an ensemble of decision trees, generally trained via bagging method, typically with <code>max_samples</code> set to the size of the training set. Instead of building a BaggingClassifier and passing it a DecisionTreeClassifier, we can use the RandomForestClassifier class, which is more convenient and optimized for Decision Trees.<br>
The Random Forest algorithm introduces extra randomness when growing trees; instead of searching for the very best feature when splitting a node (see Chapter 6), it searches for the best feature among a random subset of features. The algorithm results in greater tree diversity, which (again) trades a higher bias for a lower variance, generally yielding an overall better model.

### Extra-Trees

When you are growing a tree in a Random Forest, at each node only a random subset of
the features is considered for splitting (as discussed earlier). It is possible to make trees
even more random by also using random thresholds for each feature rather than
searching for the best possible thresholds (like regular Decision Trees do).<br>A forest of such extremely random trees is called an Extremely Randomized Trees
ensemble (or Extra-Trees for short). Once again, this technique trades more bias for a
lower variance. <br>You can create an Extra-Trees classifier using Scikit-Learn’s ExtraTreesClassifier
class.

### Feature Importance

Scikit-Learn measures a feature’s importance by
looking at how much the tree nodes that use that feature reduce impurity on average
(across all trees in the forest). More precisely, it is a weighted average, where each
node’s weight is equal to the number of training samples that are associated with it.<br>
Scikit-Learn computes this score automatically for each feature after training, then it
scales the results so that the sum of all importances is equal to 1. You can access the
result using the feature_importances_ variable.

## <font color=blue>Boosting</font>

Boosting refers to any ensemble method that can combine several weak learners into a strong learner. The general idea of most boosting methods is to train predictors sequentially, each trying to correct to correct its predecessor.

### AdaBoosting

AdaBoosting is a boosting method which a new predictor pays more attention to the training instances that are underfitted by it's predecesor. This results in new predictors focusing more and more on the hard cases. For eaxample, an AdaBoost classifier first trains a base classifier like a decision tree. The algorithm then increases the relative weight of misclassified training instances. Then it trains a new classifier using the updated weights, and so on.<br>
In AdaBoost each instance weight $w^i$ is initially set to $\frac{1}{m}$. After training the first predictor, it's weighted error rate $r_1$ is computed on the training set:

$$r_{j}=\frac{\sum_{i=1 \atop{\hat{y}_{j}^{(i)} \neq y^{(i)}}}^{m} w^{(i)}}{\sum_{i=1}^{m} w^{(i)}} \text{where } \hat{y}_{j}^{(i)} \text { is the } j^{\text {th }} \text{ predictor's prediction for the } i^{\text {th }} \text{ instance.}$$

The predictor's weight $\alpha_j$  is the computed using the following equation, where $\eta$ is the learning rate hyperparameter. The more accurate the predictor is, the higher its weight will be.

$$
\alpha_j = \eta\log{\frac{1-r_j}{r_j}}
$$

Next, instance weights are updated, using this equation, which boosts the weights of the misclassified samlpes:

$$\text{for } i = 1,2,... ,m\newline
w^{(i)}\leftarrow 
\left\{\begin{matrix}
 w^{(i)}&\text{if } \hat{y}_j^{(i)}=y^{(i)} \\ 
 w^{(i)}\exp{(\alpha_j)}& \text{if }\hat{y}_j^{(i)} \neq {y^{(i)}}
\end{matrix}\right.
$$
Then all the weights are normalized (i.e., divided by $\sum_{i=1}^{m}w^{(i)}$).
<br>
Finally, a new predictor is trained using the new weigths, and the whole process is repeated. The algorithm stops when the desired number of predictors is reached, or when a perfect predictor is found.<br>
To make predictions, AdaBoost simply computes the predictions of all the predictors
and weighs them using the predictor weights $\alpha_j$.
$$\hat{y}({X}) = \operatorname*{arg\,max}_k \sum_{j=1 \atop {\hat{y}_j(x)=k}}^N \alpha_j \text{where } N \text{is the number of predictors.}
$$
<br>
AdaBoost is availbale in Scikit-Learn ensemble package. It uses a multiclass version of Adaboost called <i>SAMME</i>. Also if the base predictor can estimate probabilities, Scikit can use <i>SAMME.R</i> (R for "Real") which depends on class probabilities rather than predictions.

### Gradient Boosting

Gradient Boosting, like AdaBoost, works by sequentially adding predictors to an ensemble, each one correcting it's predecessor. However, instead of tweaking instance weights at each iteration, this method tries to fit the new predictor to the residual errors made by the previous predictor. It first trains a base predictor on the training set, then predicts the training set with the same predictor and measures the absolute errors made by this predictor for each instance. The second predictor is then trained with these errors as target values. Third predctor is again trained on the second predictor errors and so on. The algorithm stops when it reaches the maximum number of estimators or it can be controlled with early stopping. 
<br>
Gradient boosting can be used in regression and classification tasks. When performing regression with decision trees and gradient boosting, the algorithm is called <i>Gradient Tree Boosting</i>, or <i>Gradient Boosted Regression Trees</i> (GBRT). Gradient Boosting regression and classification is available in scikit-learn with trees as base predictors. It has hyperparameters to control the growth of decision trees (e.g., max_depth), as well as hyperparameters to control the ensemble, such as number of trees (<code>n_estimators</code>).<br>The GradientBoostingRegressor class also supports a subsample hyperparameter,
which specifies the fraction of training instances to be used for training each tree. For
example, if subsample=0.25 , then each tree is trained on 25% of the training instances,
selected randomly. As you can probably guess by now, this technique trades a higher
bias for a lower variance. It also speeds up training considerably. This is called
Stochastic Gradient Boosting.<br>
It is worth noting that an optimized implementation of Gradient Boosting is available in
the popular Python library XGBoost, which stands for Extreme Gradient Boosting. This
package was initially developed by Tianqi Chen as part of the Distributed (Deep)
Machine Learning Community (DMLC), and it aims to be extremely fast, scalable, and
portable.


Check out the book and it's notebooks for code examples and more info.

## <font color=blue>Stacking</font>

Ensemble methods we discussed in the first of chapter like voting classfiers, usually use trivial aggregation functions to aggregate all the predictions. Stacking is based on a simple idea: instead of using trivial functions, why don't we train a model to perform this aggregateion? In such an ensemble, there is a layer of all the base predictors and the aggregation function which is called a <i>blender</i>, or a <i>meta laerner</i> is in another layer.<br><br>
To train a blender, a commom approach is to use a hold-out set. First, the training set is split into two subsets. One used two train the predictors in the first layer. Next, the first layer's predictors are used to make predictions on the second (held-out) set. For each instance in the hold-out set and each predictor, there is a predicted value that can be used to create a new training set with these predicted values as features and same target values as in the main train set. The blender is trained on this new training set, so it learns to predict the target value, given the first layer’s predictions.<br><br>
It is possible to train several different blenders this way, to get a whole layer of blenders with a third layer  which aggregates blenders' predictions. The trick here is to split the training set into three subsets, each for training one layer. Once this is done, we can make a prediction for a new instance by going through each layer sequentially.
<br><br>
Scikit-Learn doesn't support stacking but it is not too hard to implement one. Alternatively, we can use some other implementations such as <i>brew</i>.